In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
rows_interval = 4
rows_min = rows_interval
rows_max = 16
rows = int(rows_max/rows_interval) - int(rows_min/rows_interval) + 1

row_space = list(range(rows_min, rows_max + 1, rows_interval))
scalar_space = [1000, 10000, 100000]

columns = 3

In [ ]:
operation = "op_n"
precision = 32
median_array_op_n_32 = np.zeros((rows, columns))

# load data
for i, n in enumerate(row_space):
    for j, m in enumerate([n*1000, n*10000, n*100000]):
        try:
            directoryname = f"../blas_results/gemm_{operation}_{precision}/{n}_{m}"
            filename = "results.csv"
            df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
            subtracted_df = df["EndNs"] - df["BeginNs"]
            subtracted_array = subtracted_df.to_numpy()
            median_array_op_n_32[i, j] = np.median(subtracted_array) / 1000
        except:
            print(n,m)

In [ ]:
operation = "op_n"
precision = 64
median_array_op_n_64 = np.zeros((rows, columns))

# load data
for i, n in enumerate(row_space):
    for j, m in enumerate([n*1000, n*10000, n*100000]):
        try:
            directoryname = f"../blas_results/gemm_{operation}_{precision}/{n}_{m}"
            filename = "results.csv"
            df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
            subtracted_df = df["EndNs"] - df["BeginNs"]
            subtracted_array = subtracted_df.to_numpy()
            median_array_op_n_64[i, j] = np.median(subtracted_array) / 1000
        except:
            print(n,m)

In [ ]:
operation = "op_t"
precision = 32
median_array_op_t_32 = np.zeros((rows, columns))

# load data
for i, n in enumerate(row_space):
    for j, m in enumerate([n*1000, n*10000, n*100000]):
        try:
            directoryname = f"../blas_results/gemm_{operation}_{precision}/{n}_{m}"
            filename = "results.csv"
            df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
            subtracted_df = df["EndNs"] - df["BeginNs"]
            subtracted_array = subtracted_df.to_numpy()
            median_array_op_t_32[i, j] = np.median(subtracted_array) / 1000
        except:
            print(n,m)

In [ ]:
operation = "op_t"
precision = 64
median_array_op_t_64 = np.zeros((rows, columns))

# load data
for i, n in enumerate(row_space):
    for j, m in enumerate([n*1000, n*10000, n*100000]):
        try:
            directoryname = f"../blas_results/gemm_{operation}_{precision}/{n}_{m}"
            filename = "results.csv"
            df = pd.read_csv(f"{directoryname}/{filename}", usecols=["BeginNs", "EndNs"])
            subtracted_df = df["EndNs"] - df["BeginNs"]
            subtracted_array = subtracted_df.to_numpy()
            median_array_op_t_64[i, j] = np.median(subtracted_array) / 1000
        except:
            print(n,m)

In [ ]:
import matplotlib.patches as mpatches

titles = ["op_n_32", "op_n_64", "op_t_32", "op_t_64"]
median_arrays = [median_array_op_n_32, median_array_op_n_64, median_array_op_t_32, median_array_op_t_64]

fig, axes = plt.subplots(
    median_arrays[0].shape[1],
    median_arrays[0].shape[0], 
    subplot_kw=dict(box_aspect=1), 
    # layout="constrained",
    dpi=512,
)
fig.tight_layout()
plt.subplots_adjust(wspace=1, hspace=1)

x = np.arange(median_arrays[0].shape[0])
width = 0.25

bar_colors = ['tab:red', 'tab:orange', 'tab:blue', 'tab:purple']
lims = [100,500,4000]

for i, m in enumerate(row_space):
    for j, m_scalar in enumerate(scalar_space):
        ax = axes[len(scalar_space) - j - 1,i]
        ax.set_xlim(0, lims[j])
        ax.set_xticks(range(0,lims[j]+1, int(lims[j]/2)))
        ax.grid(which="both", axis="x")
        ms = [median_array[i][j] for median_array in median_arrays]
        rects = ax.barh(titles, ms, label=titles, color=bar_colors)
        ax.set_yticks(
            ticks=x,
            labels=ms,
            minor=False,
            fontsize="small",
        )
        ax.set_title(fr"$m = {m}$" + '\n' + fr"$n = {m*m_scalar}$")
        
# Legend
patches = []
for k, title in enumerate(titles):
    patches.append(mpatches.Patch(color=bar_colors[k], label=title))


fig.legend(handles=patches, loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=4)

plt.show()